In [1]:
from utils import db_info

import re
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

#### 데이터베이스 접속
- data.ini를 사용해 hide variable 진행

In [2]:
connection_format = f'postgresql://{db_info["USER"]}:{db_info["PASSWORD"]}@{db_info["HOST"]}:{db_info["PORT"]}/{db_info["DB"]}'
        
client = create_engine(connection_format)

#### Clinical Note에 대한 데이터 로드
- 개수는 총 23개 이다.

In [3]:
note_df = pd.read_sql('select * from de.clinical_note;', con=client)
print(len(note_df))

23


#### Note에서 person 정보를 추출하는 함수

In [4]:
def input_person(text):
    person_data = {}
    
    # person_id 중복여부 검사
    person_num = np.random.choice(len(note_df), 1)
    while person_num in person_df['person_id'].values:
        person_num = np.random.choice(len(note_df), 1)

    person_data['person_id'] = int(person_num)
    
    # birth_date 추출 후 split을 사용한 년, 월, 일 구분
    birth_date = re.findall('Birth Date:\s+?([0-9]{4}-[0-9]{2}-[0-9]{2})', text)[0]
    person_data['year_of_birth'] = birth_date.split('-')[0]
    person_data['month_of_birth'] = birth_date.split('-')[1]
    person_data['day_of_birth'] = birth_date.split('-')[2]
    
    # 성별, 인종, 민족성에 대한 정보 추출
    person_data['gender_value'] = re.findall('Gender:\s*(\w+)', text)[0]
    person_data['race_value'] = re.findall('Race:\s*(\w+)', text)[0]
    person_data['ethnicity_value'] = re.findall('Ethnicity:\s*([a-zA-Z-]+)', text)[0]
    
    # 해당 note에서 추출한 person의 정보 저장
    person_df.loc[len(person_df)] = person_data
    
    return person_num

#### Note에서 visit_occurrence 정보를 추출하는 함수

In [5]:
def input_visit_occurrence(text, person_id):
    # ENCOUNTER ~ MEDICATIONS 사이의 텍스트만 추출
    visit_text = text[text.find('ENCOUNTER'):text.find('MEDICATIONS')]
    
    # visit_occurrence_id에 대한 중복여부 검사
    visit_num = random.randint(1000, 1999)
    while visit_num in visit_df['visit_occurrence_id'].values:
        visit_num = random.randint(1000, 1999)
        
    visit_data = {}
    
    # 해당 note의 person_id와 visit_occurrence_id 정보 추출
    visit_data['visit_occurrence_id'] = int(visit_num)
    visit_data['person_id'] = int(person_id)
    
    # 해당 note의 visit_start_date, care_site_nm, visit_type_value 정보 추출
    visit_data['visit_start_date'] = re.findall('ENCOUNTER\s+([0-9]{4}-[0-9]{2}-[0-9]{2})', visit_text)[0]
    visit_data['care_site_nm'] = re.findall('Encounter at\s+([ a-zA-Z]+)', visit_text)[0]
    visit_data['visit_type_value'] = re.findall('Type:\s+([ a-zA-Z]+)', visit_text)[0]
    
    # 해당 note에서 추출한 visit_occurrence의 정보 저장
    visit_df.loc[len(visit_df)] = visit_data
    
    return visit_num

#### Note에서 drug_exposure 정보를 추출하는 함수

In [6]:
def input_drug_exposure(text, person_id, visit_id):
    # MEDICATIONS ~ CONDITIONS 사이의 텍스트만 추출
    drug_text = text[text.find('MEDICATIONS'):text.find('CONDITIONS')]
    
    drug_data = {}
    # 여러 개의 약처방이 가능하기에 try문을 통해 정보 추출 -> 약처방을 하지 않았다면 해당 note는 pass
    try:
        # 약을 처방한 date에 대한 정보 추출
        drug_date_list = re.findall('([0-9]{4}-[0-9]{2}-[0-9]{2})', drug_text)
        # 첫 번째 약처방부터 마지막 drug에 대한 정보 추출
        for num in range(len(drug_date_list)):
            # 해당 note의 person_id와 visit_occurrence_id 정보 추출
            drug_data['person_id'] = int(person_id)
            drug_data['visit_occurrence_id'] = int(visit_id)
            
            # drug_exposure_id에 대한 중복여부 검사
            drug_exposure_num = random.randint(2000, 2999)
            while drug_exposure_num in drug_df['drug_exposure_id'].values:
                drug_exposure_num = random.randint(2000, 2999)
                
            # drug_exposure_id와 drug_exposure_start_date 정보 추출
            drug_data['drug_exposure_id'] = int(drug_exposure_num)
            drug_data['drug_exposure_start_date'] = drug_date_list[num]
            
            # 해당 처방건의 drug_value 추출
            drug_name = re.findall(drug_date_list[num]+' :\s+([a-zA-z]+)', drug_text)[0]
            drug_data['drug_value'] = drug_name
            
            # 해당 처방건의 dose_value 추출
            dose = re.findall(drug_name+'\s+(\d+)', drug_text)[0]
            drug_data['dose_value'] = dose
            
            # 해당 처방건의 unit_value 추출
            unit = re.findall(dose+'\s+([a-zA-Z]+)', drug_text)[0]
            drug_data['unit_value'] = unit
            
            # 해당 처방건의 route_value 추출
            route = re.findall(unit+'\s+([a-zA-Z]+)', drug_text)[0]
            drug_data['route_value'] = route
            
            # 해당 처방건의 drug_exposure의 정보 저장
            drug_df.loc[len(drug_df)] = drug_data
    except:
        pass

#### Note에서 condition_occurrence 정보를 추출하는 함수

In [7]:
def input_condition_occurrence(text, person_id, visit_id):
    # CONDITIONS ~ CARE PLANS 사이의 텍스트만 추출
    condition_text = text[text.find('CONDITIONS'):text.find('CARE PLANS')]
    
    condition_data = {}
    # 여러 개의 진단이 가능하기에 try문을 통해 정보 추출 -> 진단을 하지 않았다면 해당 note는 pass
    try:
        # 해당 note의 진단 date list 정보 추출
        condition_date_list = re.findall('([0-9]{4}-[0-9]{2}-[0-9]{2})', condition_text)
        # 해당 note의 진단명 list 정보 추출
        condition_value_list = re.findall(condition_date_list[0]+'\s+:\s+([a-zA-z \d-]+)', condition_text)
        
        # 각 진단별로 정보 추출
        for num in range(len(condition_date_list)):
            # 해당 note의 person_id와 visit_occurrence_id 정보 추출
            condition_data['person_id'] = int(person_id)
            condition_data['visit_occurrence_id'] = int(visit_id)
            
            # condition_occurrence_id에 대한 중복여부 검사
            condition_num = random.randint(3000, 3999)
            while condition_num in condition_df['condition_occurrence_id'].values:
                condition_num = random.randint(3000, 3999)
            
            # 해당 진단에 대한 condition_occurrence_id, condition_start_date, condition_value 추출
            condition_data['condition_occurrence_id'] = int(condition_num)
            condition_data['condition_start_date'] = condition_date_list[num]
            condition_data['condition_value'] = condition_value_list[num]
            
            # 해당 진단의 condition_occurrence 정보 저장
            condition_df.loc[len(condition_df)] = condition_data
    except:
        pass

#### 4가지 정보에 대한 데이터프레임 Schema 생성

In [8]:
person_df = pd.DataFrame(columns=['person_id', 'year_of_birth', 'month_of_birth', 'day_of_birth',
                                 'gender_value', 'race_value', 'ethnicity_value'])
visit_df = pd.DataFrame(columns=['visit_occurrence_id', 'person_id', 'visit_start_date',
                                'care_site_nm', 'visit_type_value'])
drug_df = pd.DataFrame(columns=['drug_exposure_id', 'person_id', 'drug_exposure_start_date',
                               'drug_value', 'route_value', 'dose_value', 'unit_value', 'visit_occurrence_id'])
condition_df = pd.DataFrame(columns=['condition_occurrence_id', 'person_id', 'condition_start_date',
                                    'condition_value', 'visit_occurrence_id'])

#### 23개의 Note에 대한 4가지의 정보 데이터 추출 후, 데이터프레임에 저장

In [9]:
for idx, note in enumerate(note_df['note']):
    print(f'Current Note Number : {idx}')
    
    # 해당 note의 CONTINUING 부분 제외
    if note.find('CONTINUING') >= 0:
        del_idx = note.find('CONTINUING')
        note = note[:del_idx]
        
    # 해당 note의 person 정보 추가 및 현재 note의 person_id 반환
    person_id = input_person(note)
    
    # 해당 note의 visit_occurrence 정보 추가 및 현재 note의 visit_occurrence_id 반환
    visit_id = input_visit_occurrence(note, person_id)
    
    # 해당 note의 drug_exposure 정보 추가
    input_drug_exposure(note, person_id, visit_id)
    
    # 해당 note의 condition_occurrence 정보 추가
    input_condition_occurrence(note, person_id, visit_id)

Current Note Number : 0
Current Note Number : 1
Current Note Number : 2
Current Note Number : 3
Current Note Number : 4
Current Note Number : 5
Current Note Number : 6
Current Note Number : 7
Current Note Number : 8
Current Note Number : 9
Current Note Number : 10
Current Note Number : 11
Current Note Number : 12
Current Note Number : 13
Current Note Number : 14
Current Note Number : 15
Current Note Number : 16
Current Note Number : 17
Current Note Number : 18
Current Note Number : 19
Current Note Number : 20
Current Note Number : 21
Current Note Number : 22


### SQLAlchemy를 사용해 Postgres DB로 데이터 저장

#### Base 클래스 선언 (모델 생성) 

In [10]:
Base = declarative_base()

#### person 테이블에 연결할 class 생성

In [11]:
class Person(Base):
    __tablename__ = 'note_person'
    person_id = Column(BIGINT, primary_key=True)
    year_of_birth = Column(Integer, nullable=False)
    month_of_birth = Column(Integer)
    day_of_birth = Column(Integer)
    gender_value = Column(String(50))
    race_value = Column(String(50))
    ethnicity_value = Column(String(50))
    
    def __init__(self, person_id, year_of_birth, month_of_birth, day_of_birth, gender_value, race_value, ethnicity_value):
        self.person_id = person_id
        self.year_of_birth = year_of_birth
        self.month_of_birth = month_of_birth
        self.day_of_birth = day_of_birth
        self.gender_value = gender_value
        self.race_value = race_value
        self.ethnicity_value = ethnicity_value
        
    def __repr__(self):
        return f'<Person {self.person_id}, {self.year_of_birth}>'

#### visit_occurrence 테이블에 연결할 class 생성

In [12]:
class VisitOccurence(Base):
    __tablename__ = 'note_visit_occurrence'
    visit_occurrence_id = Column(BIGINT, primary_key=True)
    person_id = Column(BIGINT, ForeignKey('note_person.person_id'))
    visit_start_date = Column(Date)
    care_site_nm = Column(TEXT)
    visit_type_value = Column(String(50))
    
    def __init__(self, visit_occurrence_id, person_id, visit_start_date, care_site_nm, visit_type_value):
        self.visit_occurrence_id = visit_occurrence_id
        self.person_id = person_id
        self.visit_start_date = visit_start_date
        self.care_site_nm = care_site_nm
        self.visit_type_value = visit_type_value
        
    def __repr__(self):
        return f'<VisitOccurrence {self.visit_occurrence_id}, {self.person_id}>'

#### drug_exposure 테이블에 연결할 class 생성

In [13]:
class DrugExposure(Base):
    __tablename__ = 'note_drug_exposure'
    drug_exposure_id = Column(BIGINT, primary_key=True)
    person_id = Column(BIGINT, ForeignKey('note_person.person_id'))
    drug_exposure_start_date = Column(Date)
    drug_value = Column(TEXT)
    route_value = Column(String(50))
    dose_value = Column(String(50))
    unit_value = Column(String(50))
    visit_occurrence_id = Column(BIGINT, ForeignKey('note_visit_occurrence.visit_occurrence_id'))
    
    def __init__(self, drug_exposure_id, person_id, drug_exposure_start_date, drug_value, route_value, dose_value, unit_value, visit_occurrence_id):
        self.drug_exposure_id = drug_exposure_id
        self.person_id = person_id
        self.drug_exposure_start_date = drug_exposure_start_date
        self.drug_value = drug_value
        self.route_value = route_value
        self.dose_value = dose_value
        self.unit_value = unit_value
        self.visit_occurrence_id = visit_occurrence_id
        
    def __repr__(self):
        return f'<DrugExposure {self.drug_exposure_id}, {self.person_id}, {self.visit_occurrence_id}>'

#### condition_occurrence테이블에 연결할 class 생성

In [14]:
class ConditionOccurrence(Base):
    __tablename__ = 'note_condition_occurrence'
    condition_occurrence_id = Column(BIGINT, primary_key=True)
    person_id = Column(BIGINT, ForeignKey('note_person.person_id'))
    condition_start_date = Column(Date)
    condition_value = Column(TEXT)
    visit_occurrence_id = Column(BIGINT, ForeignKey('note_visit_occurrence.visit_occurrence_id'))
    
    def __init__(self, condition_occurrence_id, person_id, condition_start_date, condition_value, visit_occurrence_id):
        self.condition_occurrence_id = condition_occurrence_id
        self.person_id = person_id
        self.condition_start_date = condition_start_date
        self.condition_value = condition_value
        self.visit_occurrence_id = visit_occurrence_id
        
    def __repr__(self):
        return f'<ConditionOccurrence {self.condition_occurrence_id}, {self.person_id}, {self.visit_occurrence_id}>'

#### 선언된 Class들과 데이터베이스의 스키마 동기화

In [15]:
Base.metadata.create_all(client)

#### 세션 생성

In [16]:
session = sessionmaker(bind=client)()

#### person_df에 저장된 데이터를 person 테이블에 Insert

In [17]:
person_datas = []
for data in person_df.iterrows():
    data = data[1].to_dict()
    person_datas.append(Person(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [18]:
session.add_all(person_datas)
session.commit()
session.rollback()

#### visit_df 저장된 데이터를 visit_occurrence 테이블에 Insert

In [19]:
visit_datas = []
for data in visit_df.iterrows():
    data = data[1].to_dict()
    visit_datas.append(VisitOccurence(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [20]:
session.add_all(visit_datas)
session.commit()
session.rollback()

#### drug_df 저장된 데이터를 drug_exposure 테이블에 Insert

In [21]:
drug_datas = []
for data in drug_df.iterrows():
    data = data[1].to_dict()
    drug_datas.append(DrugExposure(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [22]:
session.add_all(drug_datas)
session.commit()
session.rollback()

#### condition_df 저장된 데이터를 condition_occurrence 테이블에 Insert

In [23]:
condition_datas = []
for data in condition_df.iterrows():
    data = data[1].to_dict()
    condition_datas.append(ConditionOccurrence(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [24]:
session.add_all(condition_datas)
session.commit()
session.rollback()

#### 세션 종료

In [25]:
session.close()